In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%%capture
! pip install transformers

In [18]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer
import pandas as pd
import numpy as np

In [2]:
def get_score(a, b):
    if len(a.shape) == 1: a = a.unsqueeze(0)
    if len(b.shape) == 1: b = b.unsqueeze(0)

    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]
    return torch.mm(a_norm, b_norm.transpose(0, 1)) * 100

In [3]:
model = AutoModel.from_pretrained('BM-K/KoSimCSE-bert-multitask')  
tokenizer = AutoTokenizer.from_pretrained('BM-K/KoSimCSE-bert-multitask')  

In [4]:
data = pd.read_csv('./Books.csv', encoding = 'utf-8')

In [5]:
test = pd.read_csv('./Self_Improvement.csv', encoding = 'utf-8')

In [6]:
overview = test['overview'][0] #책소개를 넣어주세요.

In [19]:
def recommend(overview, top = 10) :
    
    model.to(device)
    model.eval()
    tmp_score = []

    for i in range(len(data)) :
        if i % 100 == 0 :
            print(i,'/', len(data))
        tmp_sentence = [overview, data['overview'][i]]
        inputs = tokenizer(tmp_sentence, padding = True, truncation = True, return_tensors = 'pt').to(device)
        embeddings, _ = model(**inputs, return_dict = False)
        score = get_score(embeddings[0][0], embeddings[1][0])[0][0].item()
        tmp_score.append(score)

    score = sorted(tmp_score, reverse = True)
    score = [(data['title'][i], score) for i, score in enumerate(score[0:top])]

    return score

In [20]:
recommend(overview)

0 / 4176
100 / 4176
200 / 4176
300 / 4176
400 / 4176
500 / 4176
600 / 4176
700 / 4176
800 / 4176
900 / 4176
1000 / 4176
1100 / 4176
1200 / 4176
1300 / 4176
1400 / 4176
1500 / 4176
1600 / 4176
1700 / 4176
1800 / 4176
1900 / 4176
2000 / 4176
2100 / 4176
2200 / 4176
2300 / 4176
2400 / 4176
2500 / 4176
2600 / 4176
2700 / 4176
2800 / 4176
2900 / 4176
3000 / 4176
3100 / 4176
3200 / 4176
3300 / 4176
3400 / 4176
3500 / 4176
3600 / 4176
3700 / 4176
3800 / 4176
3900 / 4176
4000 / 4176
4100 / 4176


[('내가 틀릴 수도 있습니다 - 숲속의 현자가 전하는 마지막 인생 수업', 78.64453125),
 ('마흔에 읽는 니체 - 지금 이 순간을 살기 위한 철학 수업', 75.15026092529297),
 ('당신도 느리게 나이 들 수 있습니다 - 나이가 들어도 몸의 시간은 젊게', 75.15026092529297),
 ('이어령의 마지막 수업', 74.11676788330078),
 ('지적 대화를 위한 넓고 얕은 지식 1 - 한 권으로 현실 세계를 통달하는 지식 여행서', 73.75979614257812),
 ('하루 한 장 고전 수업 - 365일 인생의 내공을 기르는', 73.60955810546875),
 ('오십에 읽는 논어 - 굽이치는 인생을 다잡아 주는 공자의 말', 73.60955810546875),
 ('최재천의 공부 - 어떻게 배우며 살 것인가', 73.40428924560547),
 ('사피엔스 - 유인원에서 사이보그까지, 인간 역사의 대담하고 위대한 질문', 73.20674133300781),
 ('빅 히스토리 - 우주와 지구, 인간을 하나로 잇는 새로운 역사', 72.91230010986328)]